In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib import cm
import fastf1 as ff1
from fastf1.core import Laps
from fastf1 import utils
from fastf1 import plotting
plotting.setup_mpl()
from timple.timedelta import strftimedelta
import datetime
from datetime import datetime

In [2]:
import os

cache_dir = 'Desktop/Thesis/Documents/GitHub/ThesisF1/Cache'

if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

ff1.Cache.enable_cache(cache_dir)

In [3]:
schedule2019 = ff1.get_event_schedule(2019)
schedule2020 = ff1.get_event_schedule(2020)
schedule2021 = ff1.get_event_schedule(2021)
schedule2022 = ff1.get_event_schedule(2022)

In [4]:
schedule2022["Country"][2]

'Bahrain'

In [5]:
all_data = pd.DataFrame()

## Race 1

In [6]:
# Load race data
race_2022 = ff1.get_session(2022, 1, 'R')
race_2022.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2022.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)

# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2022.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2022["Country"][2]
data["Year"] = 2022
dati = schedule2022["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[2])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2022 = ff1.get_session(2022, 1, 'Q')
qualification1_2022.load()

# Add the best qualifications lap
fastest_laps = qualification1_2022.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2022.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2022.laps

auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2022) & (races["round"] == 1)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2022.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)


# Get information on number of previous wins at that particular race
#wins = pd.read_csv("podiums2022.csv", sep = ";")
#wins = wins[["Driver", "2"]]

#data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

#data = data.drop(["Driver"], axis = 1)

#data = data.rename(columns={'2': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2022.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
data.dropna(subset=['DriverNumber'], inplace=True)
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2022.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "59"
all_data = pd.concat([all_data, data])


core           INFO 	Loading data for Bahrain Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '55', '44', '63', '20', '77', '31', '22', '14', '24', '47', '18', '23', '3', '4', '6', '27', '11', '1', '10']
core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timi

In [7]:
data.columns

Index(['Abbreviation', 'DriverNumber', 'TeamName', 'Position', 'GridPosition',
       'Status', 'Points', 'RaceCountry', 'Year', 'AgeAtGP', 'BestQualiTime',
       'FLapPoint', 'AvgLapTime', 'SDLapTime', 'AvgSplitTime', 'AvgPitTime',
       'PitstopNo', 'HARD', 'INTERMEDIATE', 'MEDIUM', 'SOFT', 'WET', 'Engine',
       'Rain', 'Driver', 'AverageSpeed', 'MaxSpeed', 'AverageRPM', 'MaxRPM',
       'AverageThrottle', 'MaxThrottlePct', 'Brake', 'raceID'],
      dtype='object')

## Race 2

In [8]:
# Load race data
race_2022 = ff1.get_session(2022, 2, 'R')
race_2022.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2022.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)

# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2022.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2022["Country"][3]
data["Year"] = 2022
dati = schedule2022["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[3])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2022 = ff1.get_session(2022, 2, 'Q')
qualification1_2022.load()

# Add the best qualifications lap
fastest_laps = qualification1_2022.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2022.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2022.laps

#aux = aux[aux['DriverNumber'] != "22"]
#data = data[data["DriverNumber"] != "22"] 

#aux = aux[aux['DriverNumber'] != "24"]
#data = data[data["DriverNumber"] != "24"]


auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2022) & (races["round"] == 2)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2022.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
#wins = pd.read_csv("podiums2022.csv", sep = ";")
#wins = wins[["Driver", "2"]]

#data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

#data = data.drop(["Driver"], axis = 1)

#data = data.rename(columns={'2': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2022.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2022.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "60"
all_data = pd.concat([all_data, data])


core           INFO 	Loading data for Saudi Arabian Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
core        WARNING 	No lap data for driver 22
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 19 drivers: ['1', '16', '55', '11', '63', '31', '4', '10', '20', '44', '24', '27', '18', '23', '77', '14', '3', '6', '22']
core           INFO 	Loading data for Saudi Arabian Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver

## Race 3

In [9]:
# Load race data
race_2022 = ff1.get_session(2022, 3, 'R')
race_2022.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2022.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)

# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2022.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2022["Country"][4]
data["Year"] = 2022
dati = schedule2022["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[4])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2022 = ff1.get_session(2022, 3, 'Q')
qualification1_2022.load()

# Add the best qualifications lap
fastest_laps = qualification1_2022.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2022.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2022.laps

aux = aux[aux['DriverNumber'] != "55"]
data = data[data["DriverNumber"] != "55"] 

auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2022) & (races["round"] == 3)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2022.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
#wins = pd.read_csv("podiums2022.csv", sep = ";")
#wins = wins[["Driver", "2"]]

#data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

#data = data.drop(["Driver"], axis = 1)

#data = data.rename(columns={'2': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2022.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
data.dropna(subset=['DriverNumber'], inplace=True)
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2022.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "61"
all_data = pd.concat([all_data, data])


core           INFO 	Loading data for Australian Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '11', '63', '44', '4', '3', '31', '77', '10', '23', '24', '18', '47', '20', '22', '6', '14', '1', '5', '55']
core           INFO 	Loading data for Australian Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for

## Race 4

In [10]:
# Load race data
race_2022 = ff1.get_session(2022, 4, 'R')
race_2022.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2022.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)

# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2022.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2022["Country"][5]
data["Year"] = 2022
dati = schedule2022["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[5])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2022 = ff1.get_session(2022, 4, 'Q')
qualification1_2022.load()

# Add the best qualifications lap
fastest_laps = qualification1_2022.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2022.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

 

# Add average and std dev of lap times
aux = race_2022.laps

aux = aux[aux['DriverNumber'] != "55"]
data = data[data["DriverNumber"] != "55"] 

auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2022) & (races["round"] == 4)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2022.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
#wins = pd.read_csv("podiums2022.csv", sep = ";")
#wins = wins[["Driver", "2"]]

#data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

#data = data.drop(["Driver"], axis = 1)

#data = data.rename(columns={'2': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2022.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
data.dropna(subset=['DriverNumber'], inplace=True)
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2022.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "62"
all_data = pd.concat([all_data, data])


core           INFO 	Loading data for Emilia Romagna Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '4', '63', '77', '16', '22', '5', '20', '18', '23', '10', '44', '31', '24', '6', '47', '3', '14', '55']
core           INFO 	Loading data for Emilia Romagna Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached 

## Race 5

In [11]:
# Load race data
race_2022 = ff1.get_session(2022, 5, 'R')
race_2022.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2022.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)

# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2022.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2022["Country"][6]
data["Year"] = 2022
dati = schedule2022["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[6])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2022 = ff1.get_session(2022, 5, 'Q')
qualification1_2022.load()

# Add the best qualifications lap
fastest_laps = qualification1_2022.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2022.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2022.laps

aux = aux[aux['DriverNumber'] != "31"]
data = data[data["DriverNumber"] != "31"] 

auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

data
# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2022) & (races["round"] == 5)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2022.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
#wins = pd.read_csv("podiums2022.csv", sep = ";")
#wins = wins[["Driver", "2"]]

#data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

#data = data.drop(["Driver"], axis = 1)

#data = data.rename(columns={'2': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2022.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
data.dropna(subset=['DriverNumber'], inplace=True)
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2022.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "63"
all_data = pd.concat([all_data, data])


core           INFO 	Loading data for Miami Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '55', '11', '63', '44', '77', '31', '23', '18', '14', '22', '3', '6', '47', '20', '5', '10', '4', '24']
core           INFO 	Loading data for Miami Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_da

## Race 6

In [12]:
# Load race data
race_2022 = ff1.get_session(2022, 6, 'R')
race_2022.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2022.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)

# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2022.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2022["Country"][7]
data["Year"] = 2022
dati = schedule2022["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[7])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2022 = ff1.get_session(2022, 6, 'Q')
qualification1_2022.load()

# Add the best qualifications lap
fastest_laps = qualification1_2022.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2022.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

 

# Add average and std dev of lap times
aux = race_2022.laps

auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2022) & (races["round"] == 6)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2022.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
#wins = pd.read_csv("podiums2022.csv", sep = ";")
#wins = wins[["Driver", "2"]]

#data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

#data = data.drop(["Driver"], axis = 1)

#data = data.rename(columns={'2': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2022.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
data.dropna(subset=['DriverNumber'], inplace=True)
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2022.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "64"
all_data = pd.concat([all_data, data])


core           INFO 	Loading data for Spanish Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '63', '55', '44', '77', '31', '4', '14', '22', '5', '3', '10', '47', '18', '6', '20', '23', '24', '16']
core           INFO 	Loading data for Spanish Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timin

## Race 7

In [13]:
# Load race data
race_2022 = ff1.get_session(2022, 7, 'R')
race_2022.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2022.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)

# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2022.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2022["Country"][8]
data["Year"] = 2022
dati = schedule2022["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[8])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2022 = ff1.get_session(2022, 7, 'Q')
qualification1_2022.load()

# Add the best qualifications lap
fastest_laps = qualification1_2022.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2022.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2022.laps

auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2022) & (races["round"] == 7)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2022.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
#wins = pd.read_csv("podiums2022.csv", sep = ";")
#wins = wins[["Driver", "2"]]

#data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

#data = data.drop(["Driver"], axis = 1)

#data = data.rename(columns={'2': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2022.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
data.dropna(subset=['DriverNumber'], inplace=True)
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2022.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "65"
all_data = pd.concat([all_data, data])


core           INFO 	Loading data for Monaco Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
/Users/soniamuthi/opt/anaconda3/lib/python3.9/site-packages/fastf1/core.py:1411: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(new_last).reset_index(drop=True)
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['11', '55', '1', '16', '63

## Race 8

In [14]:
# Load race data
race_2022 = ff1.get_session(2022, 8, 'R')
race_2022.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2022.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)

# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2022.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2022["Country"][9]
data["Year"] = 2022
dati = schedule2022["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[9])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2022 = ff1.get_session(2022, 8, 'Q')
qualification1_2022.load()

# Add the best qualifications lap
fastest_laps = qualification1_2022.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2022.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2022.laps

aux = aux[aux['DriverNumber'] != "10"]
data = data[data["DriverNumber"] != "10"] 

auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2022) & (races["round"] == 8)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2022.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
#wins = pd.read_csv("podiums2022.csv", sep = ";")
#wins = wins[["Driver", "2"]]

#data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

#data = data.drop(["Driver"], axis = 1)

#data = data.rename(columns={'2': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2022.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
data.dropna(subset=['DriverNumber'], inplace=True)
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2022.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "66"
all_data = pd.concat([all_data, data])


core           INFO 	Loading data for Azerbaijan Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '63', '44', '10', '5', '14', '3', '4', '31', '77', '23', '22', '47', '6', '18', '20', '24', '16', '55']
core           INFO 	Loading data for Azerbaijan Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for

## Race 9

In [15]:
# Load race data
race_2022 = ff1.get_session(2022, 9, 'R')
race_2022.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2022.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)

# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2022.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2022["Country"][10]
data["Year"] = 2022
dati = schedule2022["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[10])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2022 = ff1.get_session(2022, 9, 'Q')
qualification1_2022.load()

# Add the best qualifications lap
fastest_laps = qualification1_2022.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2022.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2022.laps

#aux = aux[aux['DriverNumber'] != "31"]
#data = data[data["DriverNumber"] != "31"] 

auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2022) & (races["round"] == 9)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2022.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
#wins = pd.read_csv("podiums2022.csv", sep = ";")
#wins = wins[["Driver", "2"]]

#data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

#data = data.drop(["Driver"], axis = 1)

#data = data.rename(columns={'2': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2022.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
data.dropna(subset=['DriverNumber'], inplace=True)
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2022.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "67"
all_data = pd.concat([all_data, data])


core           INFO 	Loading data for Canadian Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '55', '44', '63', '16', '31', '77', '24', '14', '18', '3', '5', '23', '10', '4', '6', '20', '22', '47', '11']
core           INFO 	Loading data for Canadian Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for tim

## Race 10

In [16]:
# Load race data
race_2022 = ff1.get_session(2022, 10, 'R')
race_2022.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2022.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)

# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2022.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2022["Country"][11]
data["Year"] = 2022
dati = schedule2022["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[11])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2022 = ff1.get_session(2022, 10, 'Q')
qualification1_2022.load()

# Add the best qualifications lap
fastest_laps = qualification1_2022.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2022.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2022.laps

aux = aux[aux['DriverNumber'] != "24"]
data = data[data["DriverNumber"] != "24"] 

aux = aux[aux['DriverNumber'] != "23"]
data = data[data["DriverNumber"] != "23"]

aux = aux[aux['DriverNumber'] != "63"]
data = data[data["DriverNumber"] != "63"]

auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2022) & (races["round"] == 10)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2022.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
#wins = pd.read_csv("podiums2022.csv", sep = ";")
#wins = wins[["Driver", "2"]]

#data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

#data = data.drop(["Driver"], axis = 1)

#data = data.rename(columns={'2': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2022.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
data.dropna(subset=['DriverNumber'], inplace=True)
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2022.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "68"
all_data = pd.concat([all_data, data])


core           INFO 	Loading data for British Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['55', '11', '44', '16', '14', '4', '1', '47', '5', '20', '18', '6', '3', '22', '31', '10', '77', '63', '24', '23']
core           INFO 	Loading data for British Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timin

## Race 11

In [17]:
# Load race data
race_2022 = ff1.get_session(2022, 11, 'R')
race_2022.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2022.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)

# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2022.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2022["Country"][12]
data["Year"] = 2022
dati = schedule2022["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[12])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2022 = ff1.get_session(2022, 11, 'Q')
qualification1_2022.load()

# Add the best qualifications lap
fastest_laps = qualification1_2022.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2022.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2022.laps


#aux = aux[aux['DriverNumber'] != "14"]
#data = data[data["DriverNumber"] != "14"]

auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2022) & (races["round"] == 11)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2022.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
#wins = pd.read_csv("podiums2022.csv", sep = ";")
#wins = wins[["Driver", "2"]]

#data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

#data = data.drop(["Driver"], axis = 1)

#data = data.rename(columns={'2': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2022.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
data.dropna(subset=['DriverNumber'], inplace=True)
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2022.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "69"
all_data = pd.concat([all_data, data])


core           INFO 	Loading data for Austrian Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '1', '44', '63', '31', '47', '4', '20', '3', '14', '77', '23', '18', '24', '10', '22', '5', '55', '6', '11']
core           INFO 	Loading data for Austrian Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for tim

## Race 12

In [18]:
# Load race data
race_2022 = ff1.get_session(2022, 12, 'R')
race_2022.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2022.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)

# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2022.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2022["Country"][13]
data["Year"] = 2022
dati = schedule2022["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[13])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2022 = ff1.get_session(2022, 12, 'Q')
qualification1_2022.load()

# Add the best qualifications lap
fastest_laps = qualification1_2022.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2022.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2022.laps


auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2022) & (races["round"] == 12)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2022.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
#wins = pd.read_csv("podiums2022.csv", sep = ";")
#wins = wins[["Driver", "2"]]

#data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

#data = data.drop(["Driver"], axis = 1)

#data = data.rename(columns={'2': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2022.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
data.dropna(subset=['DriverNumber'], inplace=True)
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2022.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "70"
all_data = pd.concat([all_data, data])


core           INFO 	Loading data for French Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '44', '63', '11', '55', '14', '4', '31', '3', '18', '5', '10', '23', '77', '47', '24', '6', '20', '16', '22']
core           INFO 	Loading data for French Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_

## Race 13

In [19]:
# Load race data
race_2022 = ff1.get_session(2022, 13, 'R')
race_2022.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2022.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)

# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2022.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2022["Country"][14]
data["Year"] = 2022
dati = schedule2022["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[14])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2022 = ff1.get_session(2022, 13, 'Q')
qualification1_2022.load()

# Add the best qualifications lap
fastest_laps = qualification1_2022.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2022.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2022.laps


auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2022) & (races["round"] == 13)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2022.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
#wins = pd.read_csv("podiums2022.csv", sep = ";")
#wins = wins[["Driver", "2"]]

#data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

#data = data.drop(["Driver"], axis = 1)

#data = data.rename(columns={'2': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2022.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
data.dropna(subset=['DriverNumber'], inplace=True)
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2022.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "71"
all_data = pd.concat([all_data, data])


core           INFO 	Loading data for Hungarian Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '44', '63', '55', '11', '16', '4', '14', '31', '5', '18', '10', '24', '47', '3', '20', '23', '6', '22', '77']
core           INFO 	Loading data for Hungarian Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for t

## Race 14

In [20]:
# Load race data
race_2022 = ff1.get_session(2022, 14, 'R')
race_2022.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2022.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)

# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2022.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2022["Country"][15]
data["Year"] = 2022
dati = schedule2022["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[15])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2022 = ff1.get_session(2022, 14, 'Q')
qualification1_2022.load()

# Add the best qualifications lap
fastest_laps = qualification1_2022.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2022.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2022.laps

aux = aux[aux['DriverNumber'] != "44"]
data = data[data["DriverNumber"] != "44"]

aux = aux[aux['DriverNumber'] != "77"]
data = data[data["DriverNumber"] != "77"]

auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2022) & (races["round"] == 14)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2022.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
#wins = pd.read_csv("podiums2022.csv", sep = ";")
#wins = wins[["Driver", "2"]]

#data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

#data = data.drop(["Driver"], axis = 1)

#data = data.rename(columns={'2': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2022.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
data.dropna(subset=['DriverNumber'], inplace=True)
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2022.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "72"
all_data = pd.concat([all_data, data])


core           INFO 	Loading data for Belgian Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '63', '14', '16', '31', '5', '10', '23', '18', '4', '22', '24', '3', '20', '47', '6', '77', '44']
core           INFO 	Loading data for Belgian Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timin

## Race 15

In [21]:
# Load race data
race_2022 = ff1.get_session(2022, 15, 'R')
race_2022.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2022.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)

# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2022.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2022["Country"][16]
data["Year"] = 2022
dati = schedule2022["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[16])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2022 = ff1.get_session(2022, 15, 'Q')
qualification1_2022.load()

# Add the best qualifications lap
fastest_laps = qualification1_2022.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2022.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2022.laps

auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2022) & (races["round"] == 15)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2022.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
#wins = pd.read_csv("podiums2022.csv", sep = ";")
#wins = wins[["Driver", "2"]]

#data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

#data = data.drop(["Driver"], axis = 1)

#data = data.rename(columns={'2': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2022.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
data.dropna(subset=['DriverNumber'], inplace=True)
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2022.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "73"
all_data = pd.concat([all_data, data])


core           INFO 	Loading data for Dutch Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '63', '16', '44', '11', '14', '4', '55', '31', '18', '10', '23', '47', '5', '20', '24', '3', '6', '77', '22']
core           INFO 	Loading data for Dutch Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_da

## Race 16

In [22]:
# Load race data
race_2022 = ff1.get_session(2022, 16, 'R')
race_2022.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2022.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)

# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2022.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2022["Country"][17]
data["Year"] = 2022
dati = schedule2022["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[17])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2022 = ff1.get_session(2022, 16, 'Q')
qualification1_2022.load()

# Add the best qualifications lap
fastest_laps = qualification1_2022.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2022.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2022.laps

auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2022) & (races["round"] == 16)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2022.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
#wins = pd.read_csv("podiums2022.csv", sep = ";")
#wins = wins[["Driver", "2"]]

#data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

#data = data.drop(["Driver"], axis = 1)

#data = data.rename(columns={'2': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2022.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
data.dropna(subset=['DriverNumber'], inplace=True)
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2022.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "74"
all_data = pd.concat([all_data, data])


core           INFO 	Loading data for Italian Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '63', '55', '44', '11', '4', '10', '45', '24', '31', '47', '77', '22', '6', '20', '3', '18', '14', '5']
core           INFO 	Loading data for Italian Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timin

## Race 17

In [23]:
# Load race data
race_2022 = ff1.get_session(2022, 17, 'R')
race_2022.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2022.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)

# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2022.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2022["Country"][18]
data["Year"] = 2022
dati = schedule2022["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[18])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2022 = ff1.get_session(2022, 17, 'Q')
qualification1_2022.load()

# Add the best qualifications lap
fastest_laps = qualification1_2022.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2022.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2022.laps

auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2022) & (races["round"] == 17)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2022.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
#wins = pd.read_csv("podiums2022.csv", sep = ";")
#wins = wins[["Driver", "2"]]

#data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

#data = data.drop(["Driver"], axis = 1)

#data = data.rename(columns={'2': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2022.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
data.dropna(subset=['DriverNumber'], inplace=True)
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2022.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "75"
all_data = pd.concat([all_data, data])


core           INFO 	Loading data for Singapore Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['11', '16', '55', '4', '3', '18', '1', '5', '44', '10', '77', '20', '47', '63', '22', '31', '23', '14', '6', '24']
core           INFO 	Loading data for Singapore Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
session     WARNING 	Request for URL https:/

## Race 18

In [24]:
# Load race data
race_2022 = ff1.get_session(2022, 18, 'R')
race_2022.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2022.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)

# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2022.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2022["Country"][19]
data["Year"] = 2022
dati = schedule2022["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[19])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2022 = ff1.get_session(2022, 18, 'Q')
qualification1_2022.load()

# Add the best qualifications lap
fastest_laps = qualification1_2022.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2022.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2022.laps

aux = aux[aux['DriverNumber'] != "23"]
data = data[data["DriverNumber"] != "23"] 

aux = aux[aux['DriverNumber'] != "55"]
data = data[data["DriverNumber"] != "55"] 

auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2022) & (races["round"] == 18)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2022.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
#wins = pd.read_csv("podiums2022.csv", sep = ";")
#wins = wins[["Driver", "2"]]

#data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

#data = data.drop(["Driver"], axis = 1)

#data = data.rename(columns={'2': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2022.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
data.dropna(subset=['DriverNumber'], inplace=True)
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2022.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "76"
all_data = pd.concat([all_data, data])


core           INFO 	Loading data for Japanese Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '16', '31', '44', '5', '14', '63', '6', '4', '3', '18', '22', '20', '77', '24', '47', '10', '55', '23']
core           INFO 	Loading data for Japanese Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for tim

## Race 19

In [25]:
# Load race data
race_2022 = ff1.get_session(2022, 19, 'R')
race_2022.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2022.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)

# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2022.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2022["Country"][20]
data["Year"] = 2022
dati = schedule2022["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[20])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2022 = ff1.get_session(2022, 19, 'Q')
qualification1_2022.load()

# Add the best qualifications lap
fastest_laps = qualification1_2022.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2022.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2022.laps

aux = aux[aux['DriverNumber'] != "55"]
data = data[data["DriverNumber"] != "55"]

auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2022) & (races["round"] == 19)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2022.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
#wins = pd.read_csv("podiums2022.csv", sep = ";")
#wins = wins[["Driver", "2"]]

#data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

#data = data.drop(["Driver"], axis = 1)

#data = data.rename(columns={'2': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2022.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
data.dropna(subset=['DriverNumber'], inplace=True)
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2022.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "77"
all_data = pd.concat([all_data, data])


core           INFO 	Loading data for United States Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '44', '16', '11', '63', '4', '14', '5', '20', '22', '31', '24', '23', '10', '47', '3', '6', '18', '77', '55']
core           INFO 	Loading data for United States Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached da

## Race 20

In [26]:
# Load race data
race_2022 = ff1.get_session(2022, 20, 'R')
race_2022.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2022.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)

# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2022.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2022["Country"][21]
data["Year"] = 2022
dati = schedule2022["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[21])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2022 = ff1.get_session(2022, 20, 'Q')
qualification1_2022.load()

# Add the best qualifications lap
fastest_laps = qualification1_2022.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2022.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2022.laps

auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2022) & (races["round"] == 20)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2022.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
#wins = pd.read_csv("podiums2022.csv", sep = ";")
#wins = wins[["Driver", "2"]]

#data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

#data = data.drop(["Driver"], axis = 1)

#data = data.rename(columns={'2': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2022.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
data.dropna(subset=['DriverNumber'], inplace=True)
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2022.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "78"
all_data = pd.concat([all_data, data])


core           INFO 	Loading data for Mexico City Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '44', '11', '63', '55', '16', '3', '31', '4', '77', '10', '23', '24', '5', '18', '47', '20', '6', '14', '22']
core           INFO 	Loading data for Mexico City Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data f

## Race 21

In [27]:
# Load race data
race_2022 = ff1.get_session(2022, 21, 'R')
race_2022.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2022.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)

# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2022.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2022["Country"][22]
data["Year"] = 2022
dati = schedule2022["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[22])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2022 = ff1.get_session(2022, 21, 'Q')
qualification1_2022.load()

# Add the best qualifications lap
fastest_laps = qualification1_2022.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2022.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2022.laps

aux = aux[aux['DriverNumber'] != "3"]
data = data[data["DriverNumber"] != "3"]

aux = aux[aux['DriverNumber'] != "20"]
data = data[data["DriverNumber"] != "20"]

auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2022) & (races["round"] == 21)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2022.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
#wins = pd.read_csv("podiums2022.csv", sep = ";")
#wins = wins[["Driver", "2"]]

#data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

#data = data.drop(["Driver"], axis = 1)

#data = data.rename(columns={'2': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2022.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
data.dropna(subset=['DriverNumber'], inplace=True)
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2022.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "79"
all_data = pd.concat([all_data, data])


core           INFO 	Loading data for São Paulo Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['63', '44', '55', '16', '14', '1', '11', '31', '77', '18', '5', '24', '47', '10', '23', '6', '22', '4', '20', '3']
core           INFO 	Loading data for São Paulo Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for t

## Race 22

In [28]:
# Load race data
race_2022 = ff1.get_session(2022, 22, 'R')
race_2022.load(telemetry = True, laps = True, weather = True)

# Create dataframe
aux = race_2022.results
data = aux[["Abbreviation", "DriverNumber", "TeamName", "Position", "GridPosition", "Status", "Points"]]
data = pd.DataFrame(data)

# Add driver age at the time of the race
birth = pd.read_csv("BIRTH2022.csv", sep = ";")
birth["Birth"] = pd.to_datetime(birth["Birth"], format="%d.%m.%Y")
birth.columns = ["Abbreviation", "Birth"]

# Add race country and year
data["RaceCountry"] = schedule2022["Country"][23]
data["Year"] = 2022
dati = schedule2022["EventDate"]
data["RaceDate"] = pd.to_datetime(dati[23])
#formatted_date = timestamp.strftime('%d.%m.%Y')
data = pd.merge(data, birth, on='Abbreviation', how='inner')
data['AgeAtGP'] = (data["RaceDate"] - data["Birth"]) // pd.Timedelta(days=365.25)
data = data.drop(columns=["Birth", "RaceDate"])

# Load qualifications data
qualification1_2022 = ff1.get_session(2022, 22, 'Q')
qualification1_2022.load()

# Add the best qualifications lap
fastest_laps = qualification1_2022.laps.groupby("DriverNumber")["LapTime"].min()
fastest_laps = pd.DataFrame(fastest_laps)
fastest_laps["DriverNumber"] = fastest_laps.index
fastest_laps = fastest_laps.reset_index(drop=True)
data = pd.merge(left=data, right=fastest_laps, how='right')
data = data.rename(columns = {'LapTime': 'BestQualiTime'})

# Add fastest lap point
aux = race_2022.laps.pick_fastest()
auxDriverNo = aux[1]
data['FLapPoint'] = (data['DriverNumber'] == auxDriverNo).astype(int)

# Add average and std dev of lap times
aux = race_2022.laps

auxi = aux[["DriverNumber", "LapTime"]]
auxi

mean_lap_time = auxi.groupby('DriverNumber')['LapTime'].mean()
mean_lap_time = pd.DataFrame(mean_lap_time)

mean_lap_time["DriverNumber"] = mean_lap_time.index
mean_lap_time = mean_lap_time.reset_index(drop=True)

data = pd.merge(left=data, right=mean_lap_time, how='right')
data = data.rename(columns = {'LapTime': 'AvgLapTime'})

auxi = aux[["DriverNumber", "LapTime"]]
auxi['LapTime_ms'] = auxi['LapTime'].astype('timedelta64[ms]')
auxi = auxi.groupby(["DriverNumber"])["LapTime_ms"].std()
auxi = pd.DataFrame(auxi)

auxi["DriverNumber"] = auxi.index
auxi = auxi.reset_index(drop=True)

data = pd.merge(left=data, right=auxi, how='right')
data = data.rename(columns = {'LapTime_ms': 'SDLapTime'})
    
import datetime
data['SDLapTime'] = data['SDLapTime'].apply(lambda x: datetime.timedelta(milliseconds=x))

# Add average split times variable
besttime = pd.DataFrame(aux.groupby("LapNumber")["LapTime"].min())

besttime["LapNumber"] = besttime.index
besttime  = besttime .reset_index(drop=True)

best_lap = besttime.groupby('LapNumber')['LapTime'].min()

merged = pd.merge(aux, best_lap, on='LapNumber', suffixes=('', '_best'))

merged['diff'] = merged['LapTime'] - merged['LapTime_best']

driver_avg_diff = merged.groupby('DriverNumber')['diff'].mean().reset_index()

driver_avg_diff = driver_avg_diff.rename(columns={'diff': 'AvgSplitTime'})

data = pd.merge(left=data, right=driver_avg_diff, how='right')

# Get information about pit stops
# Add average pitstop time and number of pit stops
pit_stops = pd.read_csv("pit_stops.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")

listdrivers = np.unique(data["Abbreviation"])

filtered_drivers = drivers[drivers['code'].isin(listdrivers)]

k = filtered_drivers["driverId"]

b = races[(races["year"] == 2022) & (races["round"] == 22)]
b = b["raceId"]
b = int(b)

auxpit = pit_stops[pit_stops["driverId"].isin(k)]
auxpit = auxpit[auxpit["raceId"] == b]
auxpit = auxpit.drop(["lap", "time", "duration"], axis = 1)

merged_df = auxpit.merge(drivers, on='driverId')
merged_df = pd.DataFrame(merged_df)
merged_df = merged_df.drop(["raceId", "driverId", "driverRef", "forename", "surname", "dob", "nationality", "url"], axis = 1)

transformed_df = merged_df.groupby('code').agg({
'milliseconds': 'mean',
'stop': 'max'
}).reset_index()

transformed_df = pd.DataFrame(transformed_df)

merged_data = data.merge(transformed_df, left_on='Abbreviation', right_on='code', how = "left")

merged_data = pd.DataFrame(merged_data)
merged_data = merged_data.drop("code", axis = 1)

mask = merged_data['milliseconds'].notna()
merged_data.loc[mask, 'milliseconds'] = merged_data.loc[mask, 'milliseconds'].apply(lambda x: datetime.timedelta(milliseconds=x))

merged_data = merged_data.rename(columns = {'milliseconds': 'AvgPitTime', "stop": "PitstopNo"})

# Add number of laps on each type of compound
compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

drivers2 = aux['DriverNumber'].unique()
compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
all_combinations = pd.MultiIndex.from_product([drivers2, compounds], names=['DriverNumber', 'Compound'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()

compound_counts = aux.groupby(['DriverNumber', 'Compound']).size().reset_index(name='Laps')

compound_counts = pd.merge(all_combinations_df, compound_counts, on=['DriverNumber', 'Compound'], how='outer').fillna(0)
compound_counts_pivot = compound_counts.pivot(index='DriverNumber', columns='Compound', values='Laps').fillna(0)
indexi = compound_counts_pivot.index
indexi = list(indexi)

compound_counts_pivot = compound_counts_pivot.reset_index(level=0, drop=True)
compound_counts_pivot["DriverNumber"] = indexi
compound_counts_pivot = pd.DataFrame(compound_counts_pivot)

merged_data_compound = pd.merge(merged_data, compound_counts_pivot, on='DriverNumber')

data = merged_data_compound

# Add engine manufacturer for each team
engines = pd.read_csv("engines2022.csv", sep = ";")

data = data.merge(engines, left_on='TeamName', right_on='Car', how = "left")
data = data.drop(["Car"], axis = 1)

# Get information on number of previous wins at that particular race
#wins = pd.read_csv("podiums2022.csv", sep = ";")
#wins = wins[["Driver", "2"]]

#data = pd.merge(left=data, right=wins , how='right', left_on="Abbreviation", right_on = "Driver")

#data = data.drop(["Driver"], axis = 1)

#data = data.rename(columns={'2': 'PrevPodiumRace'})

# Get weather data
rainy = pd.DataFrame(race_2022.weather_data)

counts = rainy['Rainfall'].value_counts()

def determine_rain(counts):
    if counts.get(False, 0) > counts.get(True, 0):
        return False
    else:
        return True

is_rainy = determine_rain(counts)

data["Rain"] = is_rainy

# Add telemtry variables
data.dropna(subset=['DriverNumber'], inplace=True)
driversnumbers = np.unique(data["DriverNumber"])

agg = pd.DataFrame()
for i in driversnumbers:
    tele = pd.DataFrame(race_2022.car_data[i])
    avg_speed = tele['Speed'].mean()
    max_speed = tele['Speed'].max()
    avg_rpm = tele['RPM'].mean()
    max_rpm = tele['RPM'].max()
    avg_throttle = tele['Throttle'].mean()
    max_throttle = tele['Throttle'].max()
    max_throttle_pct = (tele['Throttle'] == max_throttle).sum() / len(tele) * 100
    break_percentage = tele['Brake'].sum() / len(tele) * 100

    driver_data = {
        'Driver': i,
        'AverageSpeed': avg_speed,
        'MaxSpeed': max_speed,
        'AverageRPM': avg_rpm,
        'MaxRPM': max_rpm,
        'AverageThrottle': avg_throttle,
        'MaxThrottlePct': max_throttle_pct,
        'Brake': break_percentage
        }
    
    agg = pd.concat([agg, pd.DataFrame(driver_data, index=[0])], ignore_index=True)

data = pd.merge(left=data, right=agg , how='right', left_on="DriverNumber", right_on = "Driver")
data["raceID"] = "80"
all_data = pd.concat([all_data, data])


core           INFO 	Loading data for Abu Dhabi Grand Prix - Race [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '11', '55', '63', '4', '31', '18', '3', '5', '22', '24', '23', '10', '77', '47', '20', '44', '6', '14']
core           INFO 	Loading data for Abu Dhabi Grand Prix - Qualifying [v2.3.0]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for t

## Complete missing values

In [29]:
# no pit stops
all_data['AvgPitTime'] = all_data['AvgPitTime'].fillna(pd.Timedelta("0 days"))

all_data['PitstopNo'] = all_data['PitstopNo'].fillna(0)

In [30]:
all_data.to_csv(r'2022data.csv', index=True, header=True)